In [ ]:
import math
import numpy as np
import gurobipy as gp


from collections import namedtuple
from sklearn import tree
from gurobipy import GRB
from scipy import stats #bruges til stats.mode
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


class OCT:

  def __init__(self, max_depth, min_samples_leaf, alpha, warmstart=True, timelimit=600, output = True):
    self.max_depth = max_depth
    self.min_samples_leaf = min_samples_leaf                              # minimum antal datapunkter i enhver slutnode
    self.alpha = alpha
    self.warmstart = warmstart
    self.output = output
    self.timelimit = timelimit
    self.trained = False
    self.optgap = None

#Indekserer over nodes
    self.index_nodes = [t+1 for t in range(2**(self.max_depth + 1) -1)]                                         # Nodes
    self.index_branch_nodes = [t+1 for t in range(math.floor(max(self.index_nodes)/2))]                         # Branch nodes
    self.index_leaf_nodes = [t+1 for t in range(math.floor(max(self.index_nodes)/2), max(self.index_nodes))]    # Leaf nodes


  def fit(self, x, y):
# Bestemmer datasættets størrelse
    self.n, self.p = x.shape           # self.n = antal datapunkter (rækker), self.p = antal features (kolonner)

    if self.output:
        print('Training data include {} instances, {} features.'.format(self.n,self.p)) #If true: Output = (observationer,indgange)

# Gemmer de unikke klasser (labels) i datasættet
    self.labels = np.unique(y)

# Skalering af data
    self.scales = np.max(x, axis=0)     #Finder den maksimale værdi for hver feature (kolonne)
    self.scales[self.scales == 0] = 1   #Hvis der findes maks-værdier på 0, erstattes de med 1 for at undgå divison med 0 senere


#Løser Mixed Integer Programming (MIP) modellen
    m, a, b, c, d, z, l = self._buildMIP(x/self.scales, y)
    if self.warmstart:
          self._setStart(x, y, a, c, d, l)
    m.optimize()
    self.optgap = m.MIPGap # MIP-gapet er forskellen mellem den bedste løsning, der er fundet indtil videre, og den bedste mulige løsning.


#Gemmer de optimerede værdier for variablerne i dictionaries
    self._a = {ind:a[ind].x for ind in a}       
    self._b = {ind:b[ind].x for ind in b}
    self._c = {ind:c[ind].x for ind in c}
    self._d = {ind:d[ind].x for ind in d}
    self._z = {ind:z[ind].x for ind in z}
    self._l = {ind:l[ind].x for ind in l}

    self.trained = True


  def predict(self, x):
       # Tjekker om modellen er trænet
        if not self.trained:
            raise AssertionError('Denne optimalDecisionTreeClassifier-instans er endnu ikke trænet.') #Hvis modellen ikke er trænet (fitted), kommer denne besked.

 # Opretter en dictionary til at mappe hver leaf node til en label
        labelmap = {}                                 #Opretter en tom dictionary til at tildele labels til hver leaf node
        for t in self.index_leaf_nodes:               #Gennemløber alle leaf nodes
            for k in self.labels:                     #Gennemløber alle mulige klasser
                if self._c[k,t] >= 1e-2:              #Hvis c[k, t] indikerer, at leaf node t har klasse k
                   labelmap[t] = k

        y_pred = []                                   ## Initialiserer en tom liste til at gemme de forudsagte labels for hvert datapunkt
        for xi in x/self.scales:                      # Gennemløber hvert datapunkt ét ad gangen                                  
            t = 1                                     #Starter i root node (node 1)
            while t not in self.index_leaf_nodes:     # Mens vi ikke er i en leaf node
                right = (sum([self._a[j,t] * xi[j] for j in range(self.p)]) + 1e-9 >= self._b[t])
                if right:
                    t = 2 * t + 1                     # Går til højre barn (ulige indeks)
            
                else:
                    t = 2 * t                         # Går til venstre barn (lige indeks)

                    # label
            y_pred.append(labelmap[t])                # Når vi er i en leaf node, gemmes den forudsagte klasse

        return np.array(y_pred)



  def _buildMIP(self, x, y):
    m = gp.Model()                            # Opretter en ny Gurobi-model m

    m.Params.outputFlag = self.output               #Bestemmer om løsningsoutput skal vises
    m.Params.LogToConsole = self.output             #Kontrolerer om logging information fra solveren vises
    # time limit
    m.Params.timelimit = self.timelimit
    # parallel
    m.params.threads = 0                            #tillader solveren at bestemme det optimale antal threads og vi maksimerer performance
    #m.params.MIPFocus=2



    m.modelSense = GRB.MINIMIZE                     # Her minimerer vi objektfunktionen


# Variables
    a = m.addVars(self.p, self.index_branch_nodes, vtype=GRB.BINARY, name='a')        # splitting feature: over indgang, branchnodes
    d = m.addVars(self.index_branch_nodes, vtype=GRB.BINARY, name='d')                # giver 1, hvis node t splitter og 0 hvis ikke
    b = m.addVars(self.index_branch_nodes, vtype=GRB.CONTINUOUS, name='b')            # højresiden, når vi splitter
    z = m.addVars(self.n, self.index_leaf_nodes, vtype=GRB.BINARY, name='z')          # leaf node assignment
    l = m.addVars(self.index_leaf_nodes, vtype=GRB.BINARY, name='l')                  # giver 1, hvis der er mindst et punkt i bladet og 0 ellers
    N = m.addVars(self.labels, self.index_leaf_nodes, vtype=GRB.CONTINUOUS, name='Nkt') # antal punkter med klasse k i blad t
    Nt = m. addVars(self.index_leaf_nodes, vtype=GRB.CONTINUOUS, name='Nt')             # samlet antal punkter i node t
    c = m.addVars(self.labels, self.index_leaf_nodes, vtype=GRB.BINARY, name='c')     # giver 1, hvis label k er givet til blad t og 0 ellers
    L = m.addVars(self.index_leaf_nodes, vtype=GRB.CONTINUOUS, name='l')              # antal datapunkter misklassificeret i blad t



    # calculate baseline accuracy
    Lhat = self._calLhat(y)                      # beregner baseline-værdien på klasse y


    # epsilon
    epsilon = self._epsilon(x)

    # objektfunktion
    obj = L.sum() / Lhat + self.alpha * d.sum()
    m.setObjective(obj)

# Bibetingelser
    #(4.1)
    m.addConstrs(a.sum('*', t) == d[t] for t in self.index_branch_nodes) #For hver branch node t skal summen af a_jt være lige d_t hvilket sikrer, at hver branch node
                                                                         # foretager præcis et split, hvir den er aktiv
    #(4.2)
    m.addConstrs(b[t] <= d[t] for t in self.index_branch_nodes) # 0 <= b_t <= d_t for alle branch nodes t
    #(4.3)---?
    m.addConstrs(d[t] <= d[t//2] for t in self.index_branch_nodes if t != 1) # En branch node kan kun være aktiv, hvis dens parent node også er aktiv.
                                                                             # Root node (t=1) har ingen parent, så den undtages, t//2 giver t/2 og så runder den ned til nærmeste heltal
    #(4.5)
    m.addConstrs(z[i,t] <= l[t] for t in self.index_leaf_nodes for i in range(self.n)) #z_it <= l_t for alle leaf nodes t og for alle i=1,..,n
    #(4.6)
    m.addConstrs(z.sum('*', t) >= self.min_samples_leaf * l[t] for t in self.index_leaf_nodes) #summen af z_it >= N_min * l_t for alle leaf nodes t
    #(4.7)
    m.addConstrs(z.sum(i, '*') == 1 for i in range(self.n)) #summen af z_it (over alle leaf nodes t) skal være lig med 1 for alle i=1,...,N (hvert punkt tildeles 1 leaf node)
    #(4.15) og (4.16)
    for t in self.index_leaf_nodes:
            left = (t % 2 == 0)           #Tjekker om t er venstre barn (lige tal) (% betyder, at restleddet efter division er 0)
            t_anc = t // 2                #Finder parent node til t
            while t_anc != 0:             #Kigger ikke på root node, da 1//2 = 0
                if left:
                    m.addConstrs(gp.quicksum(a[j,t_anc] * (x[i,j] + epsilon[j]) for j in range(self.p))
                                 +
                                 (1 + np.max(epsilon)) * (1 - d[t_anc]) 
                                 <=
                                 b[t_anc] + (1 + np.max(epsilon)) * (1 - z[i,t])
                                 for i in range(self.n))
                else:
                    m.addConstrs(gp.quicksum(a[j,t_anc] * x[i,j] for j in range(self.p))
                                 >=
                                 b[t_anc] - (1 - z[i,t])
                                 for i in range(self.n))
                left = (t_anc % 2 == 0)   #Opdater om vi er venstre eller højre barn
                t_anc //= 2               #Opdaterer til parent node for den node, man lige har behandlet, og bevæger sig på den måde op gennem træstrukturen
                                          # = denne gør at t_anc defineres på ny
    #(4.17)
    m.addConstrs(gp.quicksum((y[i] == k) * z[i,t] for i in range(self.n)) == N[k,t] for t in self.index_leaf_nodes for k in self.labels)
    #Summerer alle datapunkter med label k (y_ik) hvis datapunkt i er i leaf node t. Finder samlet antal datapunkter med label k i leaf node t
    #(2.19)
    m.addConstrs(z.sum('*', t) == Nt[t] for t in self.index_leaf_nodes) #Samlet antal punkter i leaf node t er lig med summen af z_it over i
    #(2.21)
    m.addConstrs(c.sum('*', t) == l[t] for t in self.index_leaf_nodes) #node t får kun et label k hvis den indeholder punkter, altså hvis l_t=1, ellers c_kt=0=l_t

    #For (4.23) og (4.24) er N_t=Nt[t], N_kt=N[k,t] og M=n=antal datapunkter (her self.n). Dette er lineariseringsbetingelserne
    #(4.23)
    m.addConstrs(L[t] >= Nt[t] - N[k,t] - self.n * (1 - c[k,t]) for t in self.index_leaf_nodes for k in self.labels)
    #(4.24)
    m.addConstrs(L[t] <= Nt[t] - N[k,t] + self.n * c[k,t] for t in self.index_leaf_nodes for k in self.labels)

    return m, a, b, c, d, z, l

  @staticmethod
  def _calLhat(y):
    mode = stats.mode(y)[0]                         # vælger den klasse, der går hyppigst igen i datasættet
    return np.sum(y == mode)                       # beregner baseline-værdien på klasse y



  def _epsilon(self,x):
    epsilon = []
    # Gennemløber hver feature (kolonne) i datasættet
    for j in range(x.shape[1]):
        xj = x[:,j]                                 # løber igennem alle observationer for et j
        xj = np.unique(xj)                          # Fjerner duplikater så vi kun har unikke værdier
        xj = np.sort(xj)[::-1]                      # Sorterer værdierne fra høj til lav
        dis = [1]
        for i in range(len(xj)-1):                  # for x_j gennemløbes alle observationer (over i)
          dis.append(xj[i] - xj[i+1])               # den laver en liste med alle afstandene, dvs. xj[1]-xj[2], xj[2]-xj[3]
        #Tilføjer den mindste afstand (eller 1 hvis minimum er 0) for denne feature
        epsilon.append(np.min(dis) if np.min(dis) else 1)
    return epsilon


  def _setStart(self, x, y, a, c, d, l):
        """
        set warm start from CART
        """
        # Træner et Decision Tree (CART) på data
        if self.min_samples_leaf > 1:
            clf = tree.DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
        else:
            clf = tree.DecisionTreeClassifier(max_depth=self.max_depth)
        clf.fit(x, y)

        # Henter splitting-regler fra CART træet
        rules = self._getRules(clf)               

        # Fixer startværdier for branch nodes
        for t in self.index_branch_nodes:
                                                  #Hvis den ikke splitter
            if rules[t].feat is None or rules[t].feat == tree._tree.TREE_UNDEFINED:
                d[t].start = 0                    # Sætter d[t] = 0 (noden splitter ikke)

                for j in range(self.p):
                    a[j,t].start = 0
            # Hvis noden splitter
            else:
                d[t].start = 1                    #ellers sættes d[t]=1
                for j in range(self.p):
                    if j == int(rules[t].feat):
                        a[j,t].start = 1          #a[j,t]  sættes til 1 hvis der splittes på feature j
                    else:
                        a[j,t].start = 0          #a[j,t]  sættes til 0 for alle j der ikke angiver et split

        # Fixer værdier for leaf nodes
        for t in self.index_leaf_nodes:           #gennemløber alle leaf nodes t
            # # Tjekker om vi skal afslutte tidligt (ingen datapunkter i denne node)
            if rules[t].value is None:            #Hvis værdien er None, er der ingen datapunkter i denne leaf node
                l[t].start = int(t % 2)
                # Hvis vi er på højre gren (ulige t)
                if t % 2:
                    t_leaf = t
                    while rules[t].value is None:     #hvis None, sættes l(t)=0 for lige t og l(t)=1 for ulige t
                        t //= 2
                    for k in self.labels:
                        if k == np.argmax(rules[t].value):
                            c[k, t_leaf].start = 1     #Når l(t) for leaf node t ikke er none, er der datapunkter i leaf node t og den tildeles et label
                        else:
                            c[k, t_leaf].start = 0
                # Hvis vi er på venstre gren (lige t)
                else:
                    for k in self.labels:
                        c[k, t].start = 0
            #  Hvis leaf node har datapunkter
            else:
                l[t].start = 1
                for k in self.labels:
                    if k == np.argmax(rules[t].value):
                        c[k, t].start = 1
                    else:
                        c[k, t].start = 0

  def _getRules(self, clf):
        """
        get splitting rules from a fitted CART tree
        """
        # opretter en map fra egne node-indekser(1,2,3,.....)
        node_map = {1:0}
        #gennemløber alle branch nodes i vores træ
        for t in self.index_branch_nodes:
            # antager først, at begge child nodes(venstre og højre er terminale(ingen børn))
            node_map[2*t] = -1
            node_map[2*t+1] = -1
            # Finder venstre barn i CART træet og opdaterer node_map
            l = clf.tree_.children_left[node_map[t]]
            node_map[2*t] = l
            # Finder højre barn i CART træet og opdaterer node_map
            r = clf.tree_.children_right[node_map[t]]
            node_map[2*t+1] = r


  # Opretter en struktur 'Rules', der gemmer information om split:feature, threshold og value
        rule = namedtuple('Rules', ('feat', 'threshold', 'value'))
        rules = {}
        # Gennemløber alle branch nodes
        for t in self.index_branch_nodes:
            i = node_map[t]
            if i == -1:
                #Hvis der ikke er nogen node(terminal), gemmes None for split-information
                r = rule(None, None, None)
            else:
                r = rule(clf.tree_.feature[i], clf.tree_.threshold[i], clf.tree_.value[i,0])
            rules[t] = r
        # leaf nodes
        for t in self.index_leaf_nodes:
            i = node_map[t]
            if i == -1:
                r = rule(None, None, None)
            else:
                #Ellers gemmes featur og threshold
                r = rule(None, None, clf.tree_.value[i,0])
            rules[t] = r

        return rules